In [ ]:
import os
import numpy as np
import cv2
from glob import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import *
from keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, MeanIoU
from tensorflow.keras.optimizers import Adam, Nadam, SGD
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, AveragePooling2D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from skimage import io
import tensorflow.keras.backend as K

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def dice_coef(y_true, y_pred):
    smooth=100
    y_true = tf.cast(y_true, tf.float32)
    y_truef=K.flatten(y_true)
    y_predf=K.flatten(y_pred)
    And=K.sum(y_truef* y_predf)
    return((2* And + smooth) / (K.sum(y_truef) + K.sum(y_predf) + smooth))

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


In [ ]:
def squeeze_excite_block(inputs, ratio=8):
    init = inputs
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    x = Multiply()([init, se])
    return x


In [ ]:
def stem_block(x, n_filter, strides):
    x_init = x

    ## Conv 1
    x = Conv2D(n_filter, (3, 3), padding="same", strides=strides)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(n_filter, (3, 3), padding="same")(x)

    ## Shortcut
    s  = Conv2D(n_filter, (1, 1), padding="same", strides=strides)(x_init)
    s = BatchNormalization()(s)

    ## Add
    x = Add()([x, s])
    x = squeeze_excite_block(x)
    return x

In [ ]:
def resnet_block(x, n_filter, strides=1):
    x_init = x

    ## Conv 1
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(n_filter, (3, 3), padding="same", strides=strides)(x)
    ## Conv 2
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(n_filter, (3, 3), padding="same", strides=1)(x)

    ## Shortcut
    s  = Conv2D(n_filter, (1, 1), padding="same", strides=strides)(x_init)
    s = BatchNormalization()(s)

    ## Add
    x = Add()([x, s])
    x = squeeze_excite_block(x)
    return x

In [ ]:
def aspp_block(x, num_filters, rate_scale=1):
    x1 = Conv2D(num_filters, (3, 3), dilation_rate=(6 * rate_scale, 6 * rate_scale), padding="same")(x)
    x1 = BatchNormalization()(x1)

    x2 = Conv2D(num_filters, (3, 3), dilation_rate=(12 * rate_scale, 12 * rate_scale), padding="same")(x)
    x2 = BatchNormalization()(x2)

    x3 = Conv2D(num_filters, (3, 3), dilation_rate=(18 * rate_scale, 18 * rate_scale), padding="same")(x)
    x3 = BatchNormalization()(x3)

    x4 = Conv2D(num_filters, (3, 3), padding="same")(x)
    x4 = BatchNormalization()(x4)

    y = Add()([x1, x2, x3, x4])
    y = Conv2D(num_filters, (1, 1), padding="same")(y)
    return y

In [ ]:
def attetion_block(g, x):
    """
        g: Output of Parallel Encoder block
        x: Output of Previous Decoder block
    """

    filters = x.shape[-1]

    g_conv = BatchNormalization()(g)
    g_conv = Activation("relu")(g_conv)
    g_conv = Conv2D(filters, (3, 3), padding="same")(g_conv)

    g_pool = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(g_conv)

    x_conv = BatchNormalization()(x)
    x_conv = Activation("relu")(x_conv)
    x_conv = Conv2D(filters, (3, 3), padding="same")(x_conv)

    gc_sum = Add()([g_pool, x_conv])

    gc_conv = BatchNormalization()(gc_sum)
    gc_conv = Activation("relu")(gc_conv)
    gc_conv = Conv2D(filters, (3, 3), padding="same")(gc_conv)

    gc_mul = Multiply()([gc_conv, x])
    return gc_mul

In [ ]:
class ResUnetPlusPlus:
    def __init__(self, input_size=256):
        self.input_size = input_size

    def build_model(self):
        n_filters = [16, 32, 64, 128, 256]
        inputs = Input((self.input_size, self.input_size, 3))

        c0 = inputs
        c1 = stem_block(c0, n_filters[0], strides=1)

        ## Encoder
        c2 = resnet_block(c1, n_filters[1], strides=2)
        c3 = resnet_block(c2, n_filters[2], strides=2)
        c4 = resnet_block(c3, n_filters[3], strides=2)

        ## Bridge
        b1 = aspp_block(c4, n_filters[4])

        ## Decoder
        d1 = attetion_block(c3, b1)
        d1 = UpSampling2D((2, 2))(d1)
        d1 = Concatenate()([d1, c3])
        d1 = resnet_block(d1, n_filters[3])

        d2 = attetion_block(c2, d1)
        d2 = UpSampling2D((2, 2))(d2)
        d2 = Concatenate()([d2, c2])
        d2 = resnet_block(d2, n_filters[2])

        d3 = attetion_block(c1, d2)
        d3 = UpSampling2D((2, 2))(d3)
        d3 = Concatenate()([d3, c1])
        d3 = resnet_block(d3, n_filters[1])

        ## output
        outputs = aspp_block(d3, n_filters[0])
        outputs = Conv2D(3, (1, 1), padding="same")(outputs)
        outputs = Activation("sigmoid")(outputs)

        ## Model
        model = Model(inputs, outputs)
        return model

In [ ]:
!pip install -U --no-cache-dir gdown --pre
!gdown --id 1-2HGO0znF0JAsgZ_et5Cr_owKVxr_bKZ

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=b48c8a6910e25efa5cb283f0ce383419bdd62c9f365d40af93087a15b98d0158
  Stored in directory: /tmp/pip-ephem-wheel-cache-7aaqgvmq/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be abl

In [ ]:
!gdown --id 1-2HGO0znF0JAsgZ_et5Cr_owKVxr_bKZ

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-2HGO0znF0JAsgZ_et5Cr_owKVxr_bKZ
To: /content/CT_Data.zip
100% 624M/624M [00:10<00:00, 61.4MB/s]


In [ ]:
from zipfile import ZipFile

file_name="/content/CT_Data.zip"

with ZipFile(file_name, 'r') as ziip:
  ziip.extractall()
  print('woow')

woow


In [ ]:
mask_files = glob('/content/content/drive/MyDrive/CT_Data/Masks/*')
len(mask_files)

7367

In [ ]:
CT = glob('/content/content/drive/MyDrive/CT_Data/CT_images/*')
len(CT)

7367

In [ ]:
df = pd.DataFrame({"image_path": CT, "mask_path":mask_files})
Xtrain, Xval = train_test_split(df, test_size=0.15,random_state=101)
Xtest, Xval = train_test_split(Xval, test_size=0.5)
print("Train size is {}, valid size is {} & test size is {}".format(len(Xtrain), len(Xval), len(Xtest)))

Train size is 6261, valid size is 553 & test size is 553


In [ ]:
train_ids = list(Xtrain.image_path)
train_mask = list(Xtrain.mask_path)

val_ids = list(Xval.image_path)
val_mask= list(Xval.mask_path)

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, ids , mask, image_dir = './', batch_size = 16, img_h = 256, img_w = 256, shuffle = True):
    self.ids = ids
    self.mask = mask
    self.image_dir = image_dir
    self.batch_size = batch_size
    self.img_h = img_h
    self.img_w = img_w
    self.shuffle = shuffle
    self.on_epoch_end()
  def __len__(self):
    return int(np.floor(len(self.ids)) / self.batch_size)

  def __getitem__(self, index):
    indexes = self.indexes[index* self.batch_size : (index+1) * self.batch_size]
    list_ids = [self.ids[i] for i in indexes]
    list_mask = [self.mask[i] for i in indexes]
    X, y = self.__data_generation(list_ids, list_mask)
    return X, y

  def on_epoch_end(self):    
    self.indexes = np.arange(len(self.ids))

    if self.shuffle:
      np.random.shuffle(self.indexes)

  def __data_generation(self, list_ids, list_mask):

    X = np.empty((self.batch_size, self.img_h, self.img_w, 3))
    y = np.empty((self.batch_size, self.img_h, self.img_w, 3))

    for i in range(len(list_ids)):
      img_path = str(list_ids[i])
      mask_path = str(list_mask[i])
      img = io.imread(img_path)
      mask = io.imread(mask_path)
      img = cv2.resize(img,(self.img_h,self.img_w))
      img = np.array(img, dtype = np.float64)
      mask = cv2.resize(mask,(self.img_h,self.img_w))
      mask = np.array(mask, dtype = np.float64)
      img -= img.mean()
      img /= img.std()
      mask -= mask.mean()
      mask /= mask.std()
      X[i,] = img
      y[i,] = mask
    y = (y > 0).astype(int)
    return X, y

train_data = DataGenerator(train_ids, train_mask)
val_data = DataGenerator(val_ids, val_mask)

In [ ]:
image_size = 256
batch_size = 8
lr = 0.01
epochs = 300

In [ ]:
arch = ResUnetPlusPlus(input_size=image_size)
model = arch.build_model()

In [ ]:
optimizer = Adam(lr)
metrics = [Recall(), Precision(), dice_coef, MeanIoU(num_classes=6)]
model.compile(loss=dice_coef_loss, optimizer=optimizer, metrics=metrics)

In [ ]:
filename='/content/drive/MyDrive/Res-U-Net++/history.csv'
history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)
checkpointer = ModelCheckpoint(filepath="/content/drive/MyDrive/Res-U-Net++/res-unet++.h5", 
                               verbose=1, 
                               save_best_only=True
                              )
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)

In [ ]:
callbacks = [history_logger, checkpointer, reduce_lr, early_stopping]

In [ ]:
type(val_data)

__main__.DataGenerator

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 16  64         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
history= model.fit(train_data, epochs = 300,
                  validation_data = val_data,
                   callbacks=callbacks
                 )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


Epoch 1/300
391/391 [==============================] - ETA: 0s - loss: -0.0850 - recall: 0.0487 - precision: 0.0061 - dice_coef: 0.0850 - mean_io_u: 0.5045
Epoch 1: val_loss improved from inf to -0.08006, saving model to /content/drive/MyDrive/Res-U-Net++/res-unet++.h5
391/391 [==============================] - 692s 2s/step - loss: -0.0850 - recall: 0.0487 - precision: 0.0061 - dice_coef: 0.0850 - mean_io_u: 0.5045 - val_loss: -0.0801 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_dice_coef: 0.0801 - val_mean_io_u: 0.4997 - lr: 0.0100
Epoch 2/300
391/391 [==============================] - ETA: 0s - loss: -0.0915 - recall: 0.0000e+00 - precision: 0.0000e+00 - dice_coef: 0.0915 - mean_io_u: 0.4997
Epoch 2: val_loss improved from -0.08006 to -0.10780, saving model to /content/drive/MyDrive/Res-U-Net++/res-unet++.h5
391/391 [==============================] - 657s 2s/step - loss: -0.0915 - recall: 0.0000e+00 - precision: 0.0000e+00 - dice_coef: 0.0915 - mean_io_u: 0.4997 - val_l

KeyboardInterrupt: ignored

In [ ]:
class Savefiles(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
      self.model.save_weights("Resunet++-seg-model.h5")
      model_json=self.model.to_json()
      with open("Resunet++-seg-model.json", "w") as json_file:
        json_file.write(model_json)
      !cp Resunet++-seg-model.json /content/drive/MyDrive/ResUnet50-keras
      !cp Resunet++-seg-model.h5 /content/drive/MyDrive/ResUnet50-keras
      !cp history.csv /content/drive/MyDrive/ResUnet50-keras  
save_files=Savefiles()     